In [5]:
from src.constants.constants import *
from src.feature_extraction.extract_features import *
from src.utils.helper import *
from src.models.evaluate import *
from src.models.train import *
from src.preprocessing.audio_cleaning import *
# from src.feature_extraction.save_features import *


In [6]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


In [7]:
data = pd.read_csv('csv_files/data.csv')
features = pd.read_csv('csv_files/features_40_MFCC.csv')
df = pd.merge(features, 
              data.drop(columns=['client_id', 'sentence', 'up_votes', 'down_votes', 'accent']), 
              left_on='voice_id', right_on='path', how='inner')
df.drop(columns=['path', 'Unnamed: 0'], inplace=True)
df.head()

,mean_mfcc_0,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,...,std_delta_mfcc_35,std_delta_mfcc_36,std_delta_mfcc_37,std_delta_mfcc_38,std_delta_mfcc_39,pitch,voice_id,age,gender,label
0,-177.388820,65.269430,19.547691,36.816100,-16.802723,18.517965,-7.097978,-9.195683,-10.235582,5.378500,...,2.700741,2.750742,2.837264,2.191680,2.618359,1656.88460,common_voice_en_181169.mp3,fifties,male,2
1,2.824403,124.807450,-0.138573,51.084717,-44.352936,-15.846932,-27.562298,-13.074013,-9.855357,-7.047704,...,3.839052,3.872809,3.160253,2.821814,4.303459,1285.26820,common_voice_en_122004.mp3,twenties,male,0
2,-205.324580,91.309470,-55.587543,22.580696,-37.654663,-35.335903,-48.123642,-23.904701,-12.097149,-4.848385,...,3.294928,4.633591,3.652698,2.866240,3.097598,1702.50980,common_voice_en_10208833.mp3,fifties,female,3
3,-225.895280,204.503570,-12.711844,46.873260,1.604733,-45.270008,-19.979868,-10.816832,-38.000870,-23.903002,...,2.952881,3.808742,3.526939,4.179628,3.799427,820.08923,common_voice_en_18618243.mp3,twenties,male,0
4,-0.896721,47.156086,-7.252998,89.914220,-37.487220,-46.151505,-60.026318,-0.801182,-27.701172,-15.708289,...,2.573031,3.065502,2.920276,4.575878,3.943816,1860.53880,common_voice_en_18075517.mp3,twenties,male,0


In [8]:
X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
y = df['label']

X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

In [14]:
%%time 
pipeline = get_pipeline(name='knn')
model = train_model(pipeline, X_train, y_train)
model

CPU times: user 229 ms, sys: 115 ms, total: 344 ms
Wall time: 349 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [15]:
%%time
evaluate_multi(model, X_train, y_train, type = 0)

/Users/mac/Desktop/CMP Ya Rab Ne5las /NN/project/Speaker-Age-Gender-Classification/venv/lib/python3.10/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)




nTRAINING TESTING RESULTS: 
CONFUSION MATRIX:
[[82127   470   546   287]
 [  380 12718     8   375]
 [ 1442    72 10480    58]
 [  214   495    11 10310]]
ACCURACY SCORE:
0.9637
CPU times: user 6min 8s, sys: 13.1 s, total: 6min 21s
Wall time: 47.5 s


In [16]:
%%time
evaluate_multi(model, X_val, y_val, type = 1)


\VALIDATION RESULTS: 
CONFUSION MATRIX:
[[17775   150   264   126]
 [  115  2696     5   143]
 [  455    19  2163     9]
 [   68   166     3  2184]]
ACCURACY SCORE:
0.9422
CPU times: user 1min 12s, sys: 2.4 s, total: 1min 14s
Wall time: 9.23 s


In [18]:
%%time
evaluate_multi(model, X_test, y_test, type = 2)



TESTING RESULTS: 
CONFUSION MATRIX:
[[17449   164   227   115]
 [   97  2660     3   141]
 [  460    16  2113     5]
 [   94   178     5  2097]]
ACCURACY SCORE:
0.9417
CPU times: user 1min 14s, sys: 2.51 s, total: 1min 17s
Wall time: 9.88 s
